<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/CSV_to_UVI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hier wird mit Hilfe der Messdaten der UVI berechnet

In [1]:
# Import der benötigten Module
import pandas as pd
import numpy as np


from google.colab import drive
from google.colab import files

In [2]:
# Mount google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Monat der importiert werden soll
month = '22.06'

In [4]:
# Import der CSV mit den rohen Messdaten
# Pfad zur CSV-Datei auf Google Drive
drive_path = '/content/drive/My Drive/Colab_Notebooks/CSV_Messdaten/' + month

# CSV-Datei einlesen und DataFrame erstellen
df_Messdaten = pd.read_csv(drive_path)

In [5]:
# Ausgabe des Dataframe
print(df_Messdaten)

                 Datetime       Datum   Uhrzeit  Messzeitpunkt  290.0  290.1  \
0     2022-06-15 07:21:00  2022-06-15  07:21:00          26460    0.0    0.0   
1     2022-06-15 07:23:00  2022-06-15  07:23:00          26580    0.0    0.0   
2     2022-06-15 07:25:00  2022-06-15  07:25:00          26700    0.0    0.0   
3     2022-06-15 07:27:00  2022-06-15  07:27:00          26820    0.0    0.0   
4     2022-06-15 07:29:00  2022-06-15  07:29:00          26940    0.0    0.0   
...                   ...         ...       ...            ...    ...    ...   
7303  2022-06-30 18:49:00  2022-06-30  18:49:00          67740    0.0    0.0   
7304  2022-06-30 18:51:00  2022-06-30  18:51:00          67860    0.0    0.0   
7305  2022-06-30 18:53:00  2022-06-30  18:53:00          67980    0.0    0.0   
7306  2022-06-30 18:55:00  2022-06-30  18:55:00          68100    0.0    0.0   
7307  2022-06-30 18:57:00  2022-06-30  18:57:00          68220    0.0    0.0   

      290.2  290.3  290.4  290.5  ...  

In [11]:
# calculate UVI
def calc_uvi(wavelength, spec):
    s_er = np.array(
        [(lambda x: 1 if x <= 298 else (10**(0.094*(298.0-x)) if x <= 328
                                        else 10**(0.015*(139.0-x))))(i)
         for i in wavelength])

    uvi = 40 * np.sum(np.array(spec) * s_er * 0.1)
    # uvi = 40 * uvi_temp.sum()

    return uvi

In [12]:
# Berechnen des UVI aus den Messdaten
l = np.linspace(290, 420, 1301)
df_UVI = []
# calculate for each spectrum the uvi
for line in df_Messdaten.values():
    df_UVI.append(calc_uvi(l, line))

TypeError: ignored

In [ ]:
# Berechnen des UVI aus den Messdaten


In [ ]:
# Abspeichern der Messdaten als CSV